# Duc, duc, ghost

This script scans the results of user directories to identify any that are currently out of date with our policy. These user folders are thus the data "ghosts". Once the ghosts are identified, I want to get rid of them to keep the workstations from becoming too spooky.

This works in a few steps:
1. Run shell script to list Users from nightly DUC database, currently located in `/home/patch/duc-mount-range/nightly.db`, using the `fish` command, 

    duc-list-sonic-users csv > /home/patch/Projects/sonic-user.dat

2. Run the Julia script found here to pull, sort, and filter that `.csv` for the problem direcories.

3. Do something with these results... currently could be done manually, but this will ideally include automation of alerts and deletions with a temporary backup stage that allows our users some flexibility with the changing policy, e.g. Automatic email to me, Virginia, and also the user whose data is on the machine, with notice of when the data will be "deleted". Actually, move those files to my workstation temporarily before deletion, and then delete fully on another 30-day schedule.  

Note: Step (1) currently uses the fish terminal, but in making this I learned that fish commands aren't great for Julia because `fish` isn't exactly a proper shell in Unix, or at least in the way Julia accesses it. Probably for the best in the long-run even if fish is nice for me :). TLDR; Julia can't call my fish command on its own, so I have to do it prior and thus this can't be Julia-driven without jumping through a few more hoops.

In [15]:
using CSV
using DataFrames
using Dates

df = DataFrame(CSV.File("sonic-user.dat"))
dff = filter(row -> 
    !(row.User == "AIPTadmin"
        || row.User == "Default" 
        || row.User == "Public" 
        || row.User == "Default User"),  
    df)
sort(dff)
first_of_last_month = firstdayofmonth(today() - Month(1))
ghost = dff[(dff.LastMod .< first_of_last_month), :]
sort!(ghost, [:Size], rev=true)

,LastMod,Size,Path,User
,Date,Int64,String,String15
1,2022-04-06,89829916672,/mnt/sonic/C:/Users/wfp2,wfp2
2,2022-04-06,74799718400,/mnt/sonic/C:/Users/pyousef,pyousef
3,2022-04-06,14233133056,/mnt/sonic/C:/Users/lmeier,lmeier
4,2022-04-06,14188036096,/mnt/sonic/C:/Users/jfroese,jfroese
5,2022-08-30,12817334272,/mnt/sonic/C:/Users/pema,pema
6,2022-04-06,9094795264,/mnt/sonic/C:/Users/jcoffey,jcoffey
7,2022-08-15,9063813120,/mnt/sonic/C:/Users/kulstad,kulstad
8,2022-03-31,8589856768,/mnt/sonic/C:/Users/gregoryt,gregoryt
9,2022-04-06,8414404608,/mnt/sonic/C:/Users/wwhyte,wwhyte


## Work to do...

So out of the 95 users on the machine (including the defaults and admin accounts), we have 88 folders that do not meet the criteria to keep, and this is mostly last touched over last year.

The next steps will involve removing these, and **AUTOMATION WILL HELP A LOT!!!**